In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

load the dataset and present examples

In [ ]:
dataset = pd.read_csv (r'/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
dataset.head(3)


In [ ]:
dataset.output.value_counts()
print('\n\n')
dataset.thall.value_counts()

# Get one hot encoding of columns cp
one_hot = pd.get_dummies(dataset['cp'])
one_hot.columns = [str(col) + ' cp' for col in one_hot.columns]
    
# Drop column cp as it is now encoded
dataset = dataset.drop('cp',axis = 1)
# Join the encoded df
dataset = dataset.join(one_hot)

In [ ]:
dataset.head(10)

Normalization of the data to range 0-1

In [ ]:
# no division by 0 : 
# print(dataset_Norm.max())
normalization_factors = dataset.max()
# normalize
dataset_Norm = dataset.div(dataset.max(), axis='columns', level=None, fill_value=None)


In [ ]:
print('the data frame info:')
dataset.info()

separate the data to train and validation


In [ ]:
from sklearn.model_selection import train_test_split as splt
train_db, test_db = splt(dataset_Norm, test_size=0.25, train_size=None, random_state=0, shuffle=True, stratify=None)


In [ ]:
train_db.output.value_counts()

In [ ]:
test_db.output.value_counts()

check the spliting 

In [ ]:
test_db.head(3)

In [ ]:
train_db.head(3)

In [ ]:
print('The train set info: \n')
train_db.info()

print('\n\nAnd the test set info: \n')
test_db.info()

In [ ]:
# !conda install pytorch torchvision -c pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils

Creat a data loader for pytorch

In [ ]:
batch_size = 10

train_y = torch.tensor(train_db['output'].values.astype(np.float32))
train = torch.tensor(train_db.drop('output', axis = 1).values.astype(np.float32)) 

test_y = torch.tensor(test_db['output'].values.astype(np.float32))
test = torch.tensor(test_db.drop('output', axis = 1).values.astype(np.float32)) 


train_tensor = data_utils.TensorDataset(train, train_y) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

test_tensor = data_utils.TensorDataset(test, test_y) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = True)

# print(train_tensor[:5])

# print(np.asarray(train_tensor[0][0])) # the first example
# print(np.asarray(train_tensor[0][1])) # the first examples output

Define the neural network architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input_size = 13

        hid_layer_1 = 2 ** 6
        #hid_layer_2 = 2 ** 7
        #hid_layer_3 = 2 ** 9
        #hid_layer_4 = 2 ** 7
        #hid_layer_5 = 2 ** 4

        #self.fc1 = nn.Linear(13, hid_layer_1)
        #self.fc2 = nn.Linear(hid_layer_1, hid_layer_2)
        #self.fc3 = nn.Linear(hid_layer_2, hid_layer_3)
        #self.fc4 = nn.Linear(hid_layer_3, hid_layer_4)
        #self.fc5 = nn.Linear(hid_layer_4, hid_layer_5)
        #self.fc6 = nn.Linear(hid_layer_5, 1)

        self.fc1 = nn.Linear(len(dataset.columns) - 1, hid_layer_1)
        self.fc2 = nn.Linear(hid_layer_1, hid_layer_1)
        self.fc3 = nn.Linear(hid_layer_1, hid_layer_1)
        self.fc4 = nn.Linear(hid_layer_1, hid_layer_1)
        self.fc5 = nn.Linear(hid_layer_1, hid_layer_1)
        self.fc6 = nn.Linear(hid_layer_1, 1)
        
        # dropout for the small and big matrixes
        self.dropout1 = nn.Dropout(0.5)  # 1 -> 2 ; 4 -> 5
        self.dropout2 = nn.Dropout(0.5)  # 2 -> 3 ; 3 -> 4

    def forward(self, x):
        # input through first HL. size change: 16 -> 2 ** 6
        x.type(torch.LongTensor)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        # HL1 output through 2nd HL. size change: 2 ** 6 -> 2 ** 7
        #x = F.relu(self.fc2(x))
        #x = self.dropout2(x)

        # HL2 output through 3ed HL. size change: 2 ** 6 -> 2 ** 7
        #x = F.relu(self.fc3(x))
        #x = self.dropout2(x)                            ## lets simplify the model by avoiding a Hidden Layer

        # HL3 output through 4th HL. size change: 2 ** 6 -> 2 ** 7
        #x = F.relu(self.fc4(x))
        #x = self.dropout1(x)                           ## lets simplify the model by avoiding a Hidden Layer

        # HL4 output through 5th HL. size change: 2 ** 6 -> 2 ** 7
        #x = F.relu(self.fc5(x))
        #x = self.dropout1(x)

        # HL5 output through the output unit. size change: 2 ** 6 -> 2 ** 7
        x = torch.sigmoid(self.fc6(x))
        return x
    
# initialize the NN
model = Net()
print(model)
device = 'cpu'

testing_example = torch.tensor([n*0.1 for n in range(len(dataset.columns) - 1)])
testing_example = testing_example.to(device=device)
print(model.forward(testing_example))

In [ ]:
# https://www.kaggle.com/dansbecker/running-kaggle-kernels-with-a-gpu
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    print('using device: cuda')
else:
    print('using device: cpu')
torch.cuda.is_available()

In [ ]:
## TODO: Specify loss and optimization functions
from torch import optim
# specify loss function
criterion = nn.BCELoss() # binary cross entropy loss function

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.002)
# number of epochs to train the model
n_epochs = 500  # suggest training between 20-50 epochs
rate_check = 10  # number of epochs between tests

n_tests = n_epochs // rate_check # number of tests
accuracy = np.zeros(n_tests)
print_flag = True

loss_vec = np.zeros(n_epochs)
test_loss_vec = np.zeros(n_epochs//rate_check)

for epoch in range(n_epochs):
    
    # test performance on the training every 10 epochs
    if (epoch//rate_check) * rate_check == epoch:
        with torch.no_grad():
            
            model.eval()
            test_loss = 0.0
            print(test_db.shape[0])
            predict = torch.zeros(test_db.shape[0])
            g_truth = torch.zeros(test_db.shape[0])
            idx = 0
            
            test_loss = 0.0
            
            for data, target in test_loader:
                output = model(data)
                batch_size = len(target)
                predict[idx:idx + batch_size] = torch.round( torch.reshape(output, (-1,)) )
                g_truth[idx:idx + batch_size] = target
                idx += batch_size
                
                loss = criterion(torch.reshape(output, (-1,)), target)
                test_loss += 1 * loss.item() * data.size(0)
                if print_flag: 
                    print(1*loss.item()*data.size(0))
                    
                    print('data size = '+ str(data.size(0)))
                    print_flag = False
                
            test_loss = test_loss / len(test_loader.dataset)
            test_loss_vec[epoch//rate_check] = test_loss
            
            
            correct = torch.sum(predict == g_truth)
            if print_flag:
                print('prediction:\n')
                # print(predict)
                
                print('\ng_truth:\n')
                # print(g_truth)
            test_size = len(predict)
            accuracy[epoch//rate_check] = (correct/test_size)
            print('The train accuracy is {:.3} of {} / {}'.format(accuracy[epoch//rate_check], correct, test_size))
    
    model.train()
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################

    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # uncomment to show first output and target
        if print_flag: 
            # print(output)
            # print(target)
            # print_flag = False
            pass
        
        # calculate the loss    
        loss = criterion(torch.reshape(output, (-1,)), target)
        if print_flag: 
            print(1*loss.item()*data.size(0))
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        if print_flag: 
            print(1*loss.item()*data.size(0))
            print_flag = False
        train_loss += 1*loss.item()*data.size(0)
    
    
    
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    
    loss_vec[epoch] = train_loss
    # print('Epoch: {} \tTraining Loss: {:.6f}'.format(
    #     epoch+1, 
    #     train_loss
    #     ))


visualization of the accuracy during the training

In [ ]:
from matplotlib.pyplot import plot as plt
from matplotlib.pyplot import legend
# plt(accuracy)
# print (accuracy)
print('\n\n')
plt([x/len(loss_vec) for x in range(len(loss_vec))],loss_vec)
plt([x/len(test_loss_vec) for x in range(len(test_loss_vec))],test_loss_vec)
legend(['train loss','test loss'])
